In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from mealpy import FloatVar, GWO, GA, PSO, MFO
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from warnings import filterwarnings
from scipy.spatial.distance import pdist, squareform
from scipy.special import logsumexp
from scipy.linalg import eigh
from scipy.special import logsumexp
# Disable warnings from being printed
filterwarnings('ignore')
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

#print(os.listdir("../input"))

In [4]:
df = pd.read_csv('featureoptimization/featureoptimization/densenet201_lbp_histog_feats.csv')

In [5]:
df = df[df.columns[1:]]

In [6]:
df.head()

,dense201_feature_0,dense201_feature_1,dense201_feature_2,dense201_feature_3,dense201_feature_4,dense201_feature_5,dense201_feature_6,dense201_feature_7,dense201_feature_8,dense201_feature_9,...,hist_bin_247,hist_bin_248,hist_bin_249,hist_bin_250,hist_bin_251,hist_bin_252,hist_bin_253,hist_bin_254,hist_bin_255,label
0,-0.000443,0.000154,0.008510,-0.01436,-0.001654,-0.003633,-0.107240,-0.06610,-0.001102,0.1418,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,-0.000448,0.000154,-0.006504,-0.06824,-0.001775,-0.003572,0.079100,0.14030,-0.001102,0.1503,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.000141,0.000154,-0.010620,0.01137,-0.001896,-0.003150,0.023270,-0.07810,-0.001102,0.0936,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,-0.000527,0.000154,0.031020,-0.12310,0.002342,-0.005016,-0.109200,-0.02481,-0.001102,-0.0770,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,-0.000412,0.000154,0.024730,-0.18380,0.000162,-0.006462,0.002012,-0.04570,-0.001102,0.4617,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [7]:
df.isnull().sum().max()
df.columns

Index(['dense201_feature_0', 'dense201_feature_1', 'dense201_feature_2',
       'dense201_feature_3', 'dense201_feature_4', 'dense201_feature_5',
       'dense201_feature_6', 'dense201_feature_7', 'dense201_feature_8',
       'dense201_feature_9',
       ...
       'hist_bin_247', 'hist_bin_248', 'hist_bin_249', 'hist_bin_250',
       'hist_bin_251', 'hist_bin_252', 'hist_bin_253', 'hist_bin_254',
       'hist_bin_255', 'label'],
      dtype='object', length=2433)

In [8]:
#understanding the predicted value - which is hot encoded, in real life price won't be hot encoded.
df['label'].describe(), df['label'].unique()

# there are 4 classes in the predicted value

(count    16925.000000
 mean         2.001891
 std          1.445209
 min          0.000000
 25%          1.000000
 50%          2.000000
 75%          3.000000
 max          4.000000
 Name: label, dtype: float64,
 array([2, 4, 1, 0, 3], dtype=int64))

In [9]:
y_t = np.array(df['label'])
X_t = df
X_t = df.drop(['label'],axis=1)
X_t = np.array(X_t)

In [10]:
print("shape of Y :"+str(y_t.shape))
print("shape of X :"+str(X_t.shape))

shape of Y :(16925,)
shape of X :(16925, 2432)


In [11]:
scaler = MinMaxScaler()
X_t = scaler.fit_transform(X_t)

In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(X_t,y_t,test_size=.20,random_state=42)
print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(Y_train.shape))
print("shape of Y Test :"+str(Y_test.shape))

shape of X Train :(13540, 2432)
shape of X Test :(3385, 2432)
shape of Y Train :(13540,)
shape of Y Test :(3385,)


In [13]:
def get_pca(x_train, x_test, n_components):
    pca = PCA(n_components=n_components, svd_solver='full')
    pca.fit(x_train)
    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)
    print(x_train.shape, x_test.shape)
    return x_train, x_test
     

In [12]:
temp_train_data = X_train
temp_test_data = X_test
temp_train_data, temp_test_data = get_pca(temp_train_data, temp_test_data, 0.95)

(13540, 701) (3385, 701)


In [65]:
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(temp_train_data, Y_train)
y_pred_ova = classifier.predict(temp_test_data)
print("Random_Forest_Classifier Accuracy:", accuracy_score(Y_test, y_pred_ova))

Random_Forest_Classifier Accuracy: 0.5639586410635156


In [27]:
for this_C in [1,2,3,4,5,6,7,8,9,10,11,12,13,17,18,19,20]:
    svm_ova = SVC(C = this_C,kernel='rbf', decision_function_shape='ovr')
    svm_ova.fit(temp_train_data, Y_train)
    y_pred_ova = svm_ova.predict(temp_test_data)
    print("One-vs-All Accuracy:", accuracy_score(Y_test, y_pred_ova))

One-vs-All Accuracy: 0.6676514032496307


KeyboardInterrupt: 

In [11]:
##### One-vs-One ###################################
svm_ovo = SVC(decision_function_shape='ovo')


In [19]:
svm_ovo.fit(X_train, Y_train)

SVC(decision_function_shape='ovo')

In [20]:
# Predict and evaluate the One-vs-One model
y_pred_ovo = svm_ovo.predict(X_test)
print("One-vs-One Accuracy:", accuracy_score(Y_test, Y_pred_ovo))

NameError: name 'accuracy_score' is not defined

In [23]:
print("One-vs-One Accuracy:", accuracy_score(Y_test, y_pred_ovo))

One-vs-One Accuracy: 0.9438700147710487


In [29]:
##### One-vs-all ####################
# Initialize the SVM classifier with One-vs-All strategy
svm_ova = SVC(C=7.0, gamma =0.00035, decision_function_shape='ovr')
svm_ova.fit(X_train, Y_train)

SVC(C=7.0, gamma=0.00035)

In [30]:
# Predict and evaluate the One-vs-All model
y_pred_ova = svm_ova.predict(X_test)
print("One-vs-All Accuracy:", accuracy_score(Y_test, y_pred_ova))


One-vs-All Accuracy: 0.9332348596750369


In [13]:
for this_C in [1,2,3,4,5,6,7,8,9,10,11,12,13,17,18,19,20]:
    svm_ova = SVC(C = this_C,kernel='rbf', decision_function_shape='ovr')
    svm_ova.fit(temp_train_data, Y_train)
    y_pred_ova = svm_ova.predict(temp_test_data)
    print("One-vs-All Accuracy:", accuracy_score(Y_test, y_pred_ova))

One-vs-All Accuracy: 0.948301329394387
One-vs-All Accuracy: 0.9539143279172821
One-vs-All Accuracy: 0.9556868537666174
One-vs-All Accuracy: 0.9553914327917282
One-vs-All Accuracy: 0.9559822747415067
One-vs-All Accuracy: 0.9559822747415067
One-vs-All Accuracy: 0.9559822747415067
One-vs-All Accuracy: 0.9556868537666174
One-vs-All Accuracy: 0.9559822747415067
One-vs-All Accuracy: 0.9562776957163959
One-vs-All Accuracy: 0.9562776957163959
One-vs-All Accuracy: 0.9562776957163959
One-vs-All Accuracy: 0.9565731166912851
One-vs-All Accuracy: 0.9565731166912851
One-vs-All Accuracy: 0.9565731166912851
One-vs-All Accuracy: 0.9565731166912851
One-vs-All Accuracy: 0.9565731166912851


In [16]:
def fitness_function(position):
    C,gamma = position
    C =int(C)
    print("The C value = ", C)
    print("The gamma value =", gamma)  
    svm_ova = SVC(C=C, gamma = gamma, kernel='rbf', decision_function_shape='ovr')
    svm_ova.fit(temp_train_data, Y_train)
    y_pred_ova = svm_ova.predict(temp_test_data)
    Accuracy = accuracy_score(Y_test, y_pred_ova)
    print("One-vs-All Accuracy:",Accuracy )
    return Accuracy

In [17]:
problem_dict = {
    "bounds": FloatVar(lb=[5,0.01], ub=[15,0.05], name="delta"),
    "minmax": "min",
    "obj_func": fitness_function
}
model =  GA.BaseGA(epoch=5, pop_size=50, pc=0.9, pm=0.05)
g_best = model.solve(problem_dict)
print(f"Solution: {g_best.solution}, Fitness: {g_best.target.fitness}")
print(f"Solution: {model.g_best.solution}, Fitness: {model.g_best.target.fitness}")

The C value =  13
The gamma value = 0.02966861130211986


2025/03/19 12:54:01 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.


One-vs-All Accuracy: 0.9559822747415067
The C value =  11
The gamma value = 0.04567329323153102
One-vs-All Accuracy: 0.9539143279172821
The C value =  12
The gamma value = 0.04960235404995608
One-vs-All Accuracy: 0.9536189069423929
The C value =  6
The gamma value = 0.023588843299624317
One-vs-All Accuracy: 0.9559822747415067
The C value =  14
The gamma value = 0.030895781359065712
One-vs-All Accuracy: 0.9562776957163959
The C value =  5
The gamma value = 0.029723537262733538
One-vs-All Accuracy: 0.9559822747415067
The C value =  6
The gamma value = 0.01869539054416364
One-vs-All Accuracy: 0.9548005908419498
The C value =  5
The gamma value = 0.01659261508043114
One-vs-All Accuracy: 0.9553914327917282
The C value =  10
The gamma value = 0.02167251765512656
One-vs-All Accuracy: 0.9542097488921714
The C value =  12
The gamma value = 0.044129061352236036


KeyboardInterrupt: 

In [49]:
def create_cnn():
    model = tf.keras.models.Sequential()
    #--------------------------Conv2D Layer (1) ----------------------------------
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(256, 256, 3)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding = 'same'))
    
    #--------------------------Conv2D Layer (2) ----------------------------------
    
    model.add(Conv2D(32, (5, 5)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding = 'same'))
    
    #--------------------------Conv2D Layer (3) ----------------------------------
    
    model.add(Conv2D(32, (5, 5)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0))
    model.add(MaxPooling2D(pool_size=(4, 4),strides=(6,6),  padding = 'same'))
    model.add(Dropout(0.3))
    #--------------------------Dense Layers (1) ----------------------------------
    model.add(Dense(64))
    model.add(Dense(32))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation = 'sigmoid'))
    # Define the optimizer
    #optimizer = RMSprop(learning_rate=0.000116, rho=0.9, epsilon=1e-08, decay=0.0)
    optimizer = tf.keras.optimizers.SGD(0.000116)
    return model, optimizer

In [6]:
directory_train = 'APTOS/train_images'
train_csv = pd.read_csv('APTOS/train_1.csv')

In [4]:
train_labels= train_csv['diagnosis'].tolist()

In [44]:
print(train_labels)

[2, 1, 4, 0, 0, 2, 3, 0, 0, 2, 1, 2, 4, 1, 2, 0, 1, 0, 2, 4, 2, 2, 2, 2, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 1, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 0, 1, 2, 0, 0, 2, 2, 2, 2, 0, 0, 1, 0, 4, 0, 2, 0, 0, 4, 2, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 4, 0, 2, 1, 4, 0, 0, 2, 2, 2, 2, 0, 2, 4, 0, 0, 0, 4, 2, 0, 2, 0, 4, 4, 1, 0, 0, 2, 4, 1, 2, 0, 2, 0, 0, 4, 1, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 3, 0, 2, 2, 2, 0, 0, 0, 2, 2, 4, 4, 1, 0, 3, 1, 0, 0, 0, 2, 0, 2, 0, 4, 0, 1, 0, 3, 0, 4, 2, 1, 2, 0, 2, 4, 4, 0, 2, 4, 3, 0, 0, 2, 3, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 1, 0, 2, 1, 0, 2, 2, 4, 1, 0, 2, 4, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 1, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 3, 1, 0, 1, 4, 0, 2, 0, 1, 2, 0, 0, 2, 2, 0, 2, 2, 0, 1, 0, 2, 2, 2, 0, 2, 0, 0, 3, 0, 0, 2, 2, 3, 0, 4, 3, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 1, 2, 1, 0, 0, 0, 0, 4, 2, 2, 0, 1, 0, 0, 0, 0, 4, 1, 3, 1, 2, 4, 0, 0, 0, 2, 2, 4, 0, 2, 4, 0, 2, 0, 2, 1, 0, 0, 2, 1, 1, 0, 2, 4, 2, 0, 0, 2, 2, 0, 3, 0, 0, 2, 1, 

In [7]:
train_dataset, valid_dataset = tf.keras.utils.image_dataset_from_directory(
        directory_train, labels=train_labels, batch_size=32, shuffle=True, validation_split=0.2, seed=42, subset='both')

Found 2930 files belonging to 1 classes.
Using 2344 files for training.
Using 586 files for validation.


In [53]:
def train_CNN(train_dataset,valid_dataset):
    h1 = []
    CNN, optimizer = create_cnn() 
    CNN.compile(optimizer,loss='categorical_crossentropy',metrics=['acc',tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
    h1 = CNN.fit(train_dataset, batch_size=32,epochs=50,validation_data=valid_dataset)
    return h1,CNN

#h1,model = train_CNN(train_dataset,valid_dataset)
#print("The Testing Performance:------------")
#model.evaluate(Test_DS2)

In [54]:
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,n5,krnl_size,epochs,_,_,_ = position
    n1,n2,n3,n4,n5, krnl_size,epochs=int(n1),int(n2),int(n3),int(n4),int(n5), int(krnl_size),int(epochs)
    print("The Optimum values are")
    print(learning_rate,n1,n2,n3,n4,n5,krnl_size,epochs)

    model = tf.keras.models.Sequential()
    #--------------------------Conv2D Layer (1) ----------------------------------
    model.add(Conv2D(n1, (krnl_size, krnl_size), activation='relu', input_shape=(256, 256, 3)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding = 'same'))
    
    #--------------------------Conv2D Layer (2) ----------------------------------
    
    model.add(Conv2D(n2, (krnl_size, krnl_size)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2), padding = 'same'))
    
    #--------------------------Conv2D Layer (3) ----------------------------------
    
    model.add(Conv2D(n3, (krnl_size, krnl_size)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0))
    model.add(MaxPooling2D(pool_size=(4, 4),strides=(6,6),  padding = 'same'))
    #model.add(Dropout(0.1))
    #--------------------------Dense Layers (1) ----------------------------------
    model.add(Dense(n4))
    model.add(Dense(n5))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation = 'sigmoid'))
    # Define the optimizer
    #optimizer = RMSprop(learning_rate=0.000116, rho=0.9, epsilon=1e-08, decay=0.0)
    optimizer = tf.keras.optimizers.SGD(learning_rate)

    #-------------------------------------------------------------------------------------------------
    #optimizer = tf.keras.optimizers.SGD(0.000116)
    #optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(optimizer,loss='categorical_crossentropy',metrics=['acc',tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall')])
    history = model.fit(train_dataset, batch_size=32,epochs=epochs,validation_data=valid_dataset)
    return history.history['val_acc'][-1]

In [55]:
problem_dict = {
    "bounds": FloatVar(lb=[0.0001 ,28,28,28,50,1,2,30,1,1,1], ub=[0.0005,70,70,70,200,1000,7,50,1,1,1], name="delta"),
    "minmax": "min",
    "obj_func": fitness_function
}

model = GA.BaseGA(epoch=2, pop_size=5, pc=0.9, pm=0.05)
best_position, best_fitness = model.solve(problem_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")


The Optimum values are
0.00048430817022836073 47 47 38 87 653 6 47
Epoch 1/47
74/74 [==============================] - 25s 288ms/step - loss: 0.0000e+00 - acc: 0.1122 - precision: 0.5183 - recall: 0.9664 - val_loss: 0.0000e+00 - val_acc: 0.1109 - val_precision: 0.4727 - val_recall: 1.0000
Epoch 2/47
74/74 [==============================] - 22s 277ms/step - loss: 0.0000e+00 - acc: 0.1003 - precision: 0.5201 - recall: 1.0000 - val_loss: 0.0000e+00 - val_acc: 0.1109 - val_precision: 0.4727 - val_recall: 1.0000
Epoch 3/47
74/74 [==============================] - 22s 273ms/step - loss: nan - acc: 0.1817 - precision: 0.5213 - recall: 0.7925 - val_loss: nan - val_acc: 0.5273 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/47
74/74 [==============================] - 21s 255ms/step - loss: nan - acc: 0.4799 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: nan - val_acc: 0.5273 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/47
74/74 [===========================

2025/03/15 02:28:38 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.


The Optimum values are
0.00035871962683477887 65 46 32 147 322 2 32
Epoch 1/32
74/74 [==============================] - 21s 251ms/step - loss: 0.0000e+00 - acc: 0.1088 - precision: 0.5206 - recall: 0.9836 - val_loss: 0.0000e+00 - val_acc: 0.1109 - val_precision: 0.4727 - val_recall: 1.0000
Epoch 2/32
74/74 [==============================] - 22s 270ms/step - loss: 0.0000e+00 - acc: 0.1003 - precision: 0.5201 - recall: 1.0000 - val_loss: 0.0000e+00 - val_acc: 0.1109 - val_precision: 0.4727 - val_recall: 1.0000
Epoch 3/32
74/74 [==============================] - 22s 271ms/step - loss: 0.0000e+00 - acc: 0.1003 - precision: 0.5201 - recall: 1.0000 - val_loss: 0.0000e+00 - val_acc: 0.1109 - val_precision: 0.4727 - val_recall: 1.0000
Epoch 4/32
74/74 [==============================] - 20s 242ms/step - loss: nan - acc: 0.2368 - precision: 0.5183 - recall: 0.6390 - val_loss: nan - val_acc: 0.5273 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/32
74/74 [============================

KeyboardInterrupt: 